In [43]:
import pdfplumber
import pandas as pd
import re
import os
from typing import Optional

def time_to_seconds(t):
    if ':' in t:
        # Format is like '9:38.89'
        minutes, sec_millisec = t.split(':')
        minutes = int(minutes)
        seconds = float(sec_millisec)
        total_seconds = minutes * 60 + seconds
    else:
        # Format is like '38.89'
        total_seconds = float(t)
    return total_seconds

def extract_meet_title(pdf_path: str) -> str:
    with pdfplumber.open(pdf_path) as pdf:
        first_page_text = pdf.pages[0].extract_text()
        lines = first_page_text.split('\n')
        
        # Heuristic: Look for the first line that is long and looks like a meet name
        for line in lines:
            # Example: Skip very short lines or admin headers
            if len(line) > 20 and not any(x in line.lower() for x in ['license', 'page', 'organization']):
                # Clean it a little if needed (remove trailing dates)
                line = re.sub(r'- \\d{2}/\\d{2}/\\d{4}.*', '', line)
                return line.strip()
        
        return "Unknown Meet"

def extract_hytek_results(folder_path: str, output_csv: Optional[str] = None) -> pd.DataFrame:
    """
    Extract swimming event results from a list of PDF files (Hy-tek format PDF) in a folder.
    
    Args:
        pdf_path (str): Path to a folder holding the pdf files.
        output_csv (Optional[str]): If provided, saves the results to this CSV file.
    
    Returns:
        pd.DataFrame: A DataFrame containing the parsed results.
    """
    pdf_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]

    meet_dfs = []
    for pdf_path in pdf_paths:
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() + '\n'

        # Split into event sections
        event_blocks = re.split(r'(?=Event \d+)', text)
        meet_title = extract_meet_title(pdf_path)

        event_dfs = []

        for block in event_blocks:
            # Detect if prelims or finals
            swim_type = "Unknown"
            if 'Preliminaries' in block:
                swim_type = "Prelims"
            elif 'Finals' in block:
                swim_type = "Finals"
            else:
                swim_type = "Finals"  # Default assumption if not explicitly stated

            # Find the event header
            match = re.search(r'Event (\d+)\s+(.+)', block)
            if not match:
                continue
            event_number, event_title = match.groups()

            # Find swimmer lines
            swimmer_lines = re.findall(
                r'([A-Za-z &\.\'\-]+)\s+(\d+)([A-Za-z \-\&]+)\s+([\d:\.]+)\s+([\d:\.]+)',
                block
            )

            if swimmer_lines:
                df = pd.DataFrame(swimmer_lines, columns=['Name', 'Age', 'Team', 'Seed Time', 'Performance Time'])
                df['EVENT'] = event_title
                df['EVENT_NUM'] = event_number

                # Extract details from event title
                event_text = event_title.split(' ')
                if len(event_text) >= 6:  # basic check
                    df['PLACE'] = range(1, len(df) + 1)
                    df["GENDER"] = event_text[0]
                    df["AGE_GROUP"] = event_text[1]
                    df["DISTANCE"] = event_text[2]
                    df["STROKE"] = event_text[5]
                    df['DISTANCEXSTROKE'] = event_text[2] + event_text[5]
                    df["AGE_GROUPXGENDER"] = event_text[1] + event_text[0]
                    df['MEET'] = meet_title
                    df['SWIM_TYPE'] = swim_type
                    df['PCT_DROP_GAIN'] = -((df['Seed Time'].apply(time_to_seconds) - df['Performance Time'].apply(time_to_seconds)) / df['Seed Time'].apply(time_to_seconds)) * 100 

                df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
                event_dfs.append(df)

        meet_df = pd.concat(event_dfs)
        meet_dfs.append(meet_df)

    # Combine all meets into one DataFrame
    results_df = pd.concat(meet_dfs, ignore_index=True)
    results_df = results_df.map(lambda x: x.strip() if isinstance(x, str) else x)

    if output_csv:
        results_df.to_csv(output_csv, index=False)

    return results_df








In [44]:
# Example usage:
df = extract_hytek_results('result_pdfs', output_csv='carifta_2025_results.csv')
print(df.head())

/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T

     Name Age                     Team Seed Time Performance Time  \
0  Reagan  13  Virgin Islands Swimming   9:38.89          9:22.27   
1  Marena  13        Trinidad & Tobago   9:46.17          9:49.52   
2    Zara  13        Trinidad & Tobago   9:54.23          9:53.57   
3  Dndn F  14                  Bahamas  10:03.47         10:02.06   
4  Lailah  13       Barbados Swim Team  10:46.29         10:14.26   

                                EVENT EVENT_NUM  PLACE GENDER AGE_GROUP  \
0  Girls 13-14 800 LC Meter Freestyle         1      1  Girls     13-14   
1  Girls 13-14 800 LC Meter Freestyle         1      2  Girls     13-14   
2  Girls 13-14 800 LC Meter Freestyle         1      3  Girls     13-14   
3  Girls 13-14 800 LC Meter Freestyle         1      4  Girls     13-14   
4  Girls 13-14 800 LC Meter Freestyle         1      5  Girls     13-14   

  DISTANCE     STROKE DISTANCEXSTROKE AGE_GROUPXGENDER  \
0      800  Freestyle    800Freestyle       13-14Girls   
1      800  Freest

/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T/ipykernel_57731/3821424748.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(')') if isinstance(x, str) else x)
/var/folders/gx/7z1dq92j5c9gx2mq7hmzzjbh0000gn/T

In [18]:
import pdfplumber
import re

def extract_meet_title(pdf_path: str) -> str:
    with pdfplumber.open(pdf_path) as pdf:
        first_page_text = pdf.pages[0].extract_text()
        lines = first_page_text.split('\n')
        
        # Heuristic: Look for the first line that is long and looks like a meet name
        for line in lines:
            # Example: Skip very short lines or admin headers
            if len(line) > 20 and not any(x in line.lower() for x in ['license', 'page', 'organization']):
                # Clean it a little if needed (remove trailing dates)
                line = re.sub(r'- \\d{2}/\\d{2}/\\d{4}.*', '', line)
                return line.strip()
        
        return "Unknown Meet"
        
# Example usage
title = extract_meet_title('result_pdfs/carifta_2025_results.pdf')
print(title)


XXXVIII Carifta 2025 Aquatic Championships - 19/04/2025 to 22/04/2025
